In [ ]:
import os
import torch
from PIL import Image
from tqdm.auto import tqdm, trange
from torchmetrics.image import StructuralSimilarityIndexMeasure, LearnedPerceptualImagePatchSimilarity
from metrics import *
from dev import *
from utils import to_tensor

# Example paths
clean_path = "/fs/nexus-projects/HuangWM/datasets/main/diffusiondb/real"
attacked_path = "/fs/nexus-projects/HuangWM/datasets/attacked/diffusiondb/distortion_single_rotation-9-real"

# Loading all, for easy testing
clean_images = [
    Image.open(os.path.join(clean_path, f"{idx}.png")) for idx in range(LIMIT)
]
attacked_images = [
    Image.open(os.path.join(attacked_path, f"{idx}.png")) for idx in range(LIMIT)
]

In [ ]:
compute_fid(
    clean_path,
    attacked_path,
    mode="legacy",
    device=torch.device("cuda"),
    batch_size=128,
    num_workers=8,
    verbose=True,
)

compute_fid(
    clean_path,
    attacked_path,
    mode="clip",
    device=torch.device("cuda"),
    batch_size=128,
    num_workers=8,
    verbose=True,
)

In [ ]:
compute_psnr_repeated(clean_images, attacked_images, num_workers=8, verbose=True)
compute_ssim_repeated(clean_images, attacked_images, num_workers=8, verbose=True)
compute_nmi_repeated(clean_images, attacked_images, num_workers=8, verbose=True)

In [ ]:
ssim = StructuralSimilarityIndexMeasure(reduction=None).cuda()
batch_size = 32
for i in trange(0, len(clean_images), batch_size):
    clean_tensors = to_tensor(clean_images[i: min(i+batch_size, LIMIT)]).cuda()
    attacked_tensors = to_tensor(attacked_images[i: min(i+batch_size, LIMIT)]).cuda()
    ssim(clean_tensors, attacked_tensors).cpu().tolist()

In [ ]:
lpips = LearnedPerceptualImagePatchSimilarity(net_type='alex').cuda()

batch_size = 1
for i in trange(0, len(clean_images), batch_size):
    clean_tensors = to_tensor(clean_images[i: min(i+batch_size, LIMIT)]).cuda()
    attacked_tensors = to_tensor(attacked_images[i: min(i+batch_size, LIMIT)]).cuda()
    for j in range(batch_size):
        lpips(clean_tensors[j].unsqueeze(0), attacked_tensors[j].unsqueeze(0)).cpu().item()

In [ ]:
models = load_aesthetics_and_artifacts_models()

clean_aesthetics_scores, clean_artifaces_scores = [], []
for image in tqdm(clean_images):
    aesthetics_score, artifaces_score = compute_aesthetics_and_artifacts_scores(
        image, models
    )
    clean_aesthetics_scores.append(aesthetics_score)
    clean_artifaces_scores.append(artifaces_score)

In [ ]:
models = load_open_clip_model_preprocess_and_tokenizer()
prompt = "A photo of a cat"

clip_scores = []
for image in tqdm(clean_images):
    clip_score = compute_clip_score(image, prompt, models)
    clip_scores.append(clip_score)